In [1]:
import os
import math

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import spacy

# Arquivos Pythons com funções e constantes
from qs_constants import AppConstants
from qs_functions import load_qa_pairs, normalize_qa, remove_duplication, remove_bad_answer, get_context_files
from qs_classes import TitleGroup

In [2]:
nlp = spacy.load('en_core_web_lg')

In [3]:
df_all = load_qa_pairs()
df_preparado = df_all[(df_all.Question.notna()) &
                          (df_all.Answer.notna())]
vet_QA = df_preparado[["Question", "Answer", "ArticleTitle","ArticleFile"]].to_numpy()
vet_QA = normalize_qa(vet_QA, lowercase=False)
vet_QA = remove_bad_answer(vet_QA)
vet_QA = remove_duplication(vet_QA)


In [4]:
vet_QA[1]

array(['Did Lincoln sign the National Banking Act of 1863?', 'yes',
       'Abraham_Lincoln', 'S08_set3_a4'], dtype=object)

In [5]:
df_all.ArticleTitle.value_counts()

Amedeo_Avogadro     132
Alessandro_Volta    104
Michael_Faraday      84
Isaac_Newton         77
Calvin_Coolidge      77
                   ... 
Koala                 8
Nassau                8
Michelangelo          7
Vincent_van_Gogh      7
\n                    1
Name: ArticleTitle, Length: 109, dtype: int64

In [6]:
qa_agrupada_titulo = {}
for qa in vet_QA:
    titulo = qa[AppConstants.COL_TITULO]
    titulo_preparado = titulo.lower().replace("_", " ")
    
    if titulo_preparado not in qa_agrupada_titulo:
        grupo = TitleGroup()
        grupo.id = len(qa_agrupada_titulo)
        qa_agrupada_titulo[titulo_preparado] = grupo
    else:
        grupo = qa_agrupada_titulo[titulo_preparado]
    
    grupo.qa_pairs.append(qa[0:2])
    
    # existem arquivos com nan
    file = qa[AppConstants.COL_CONTEXT_FILE]
    if isinstance(file, str):
        grupo.files.add(file)
    
    
    

In [7]:
list(grupo.context_ner)

[]

In [8]:
for k_grupo in qa_agrupada_titulo:
    
    grupo = qa_agrupada_titulo[k_grupo]
    for ctx_file in grupo.files:
        print('                                   ', end='\r')
        print(k_grupo, ctx_file, end='\r')
        file_path = os.path.join(AppConstants.CONTEXT_TEXT_PATH, ctx_file + AppConstants.CONTEXT_FILE_EXTENSION)
        with open(file_path, 'r', encoding='utf-8') as f_ctx:
            ctx_file_doc = f_ctx.read()
            ctx_nlp_doc = nlp(ctx_file_doc)
            if ctx_nlp_doc.ents:
                for ent in ctx_nlp_doc.ents:
                    entidade = ent.text
                    grupo.context_ner.add(entidade)



In [9]:
x = nlp('What is the difference between Celsius\' original temperature scale and the one we use today?')
for word in x:
        print(word.text, word.pos_)

What PRON
is AUX
the DET
difference NOUN
between ADP
Celsius PROPN
' PART
original ADJ
temperature NOUN
scale NOUN
and CCONJ
the DET
one NOUN
we PRON
use VERB
today NOUN
? PUNCT


In [12]:
# imprime as perguntas cuja a entidade não conseguiu ser encontrada
total = 0
qtd_achou = 0
qtd_nao_achou = 0
estatistica_busca = {'NER_PERGUNTA': 0, 
                     'NOUNS' : 0,
                     'LEMMA' : 0,
                     'NER_CTX_FILE' : 0,
                     'PERGUNTA_SEM_CONTEXTO' : 0
                    }

for k_titulo in qa_agrupada_titulo:
    grupo = qa_agrupada_titulo[k_titulo]
    for qa in grupo.qa_pairs:
        total += 1
        achou = False
        como_achou = ""
        pergunta = qa[AppConstants.COL_PERGUNTA]
        doc = nlp(pergunta)
        
        # Procura por entidades
        if doc.ents:
            for ent in doc.ents:
                entidade = ent.text
                entidade = entidade.lower()
                for ent in entidade.split():
                    if ent in k_titulo:
                        achou = True
                        como_achou = 'NER_PERGUNTA' 
        # Prcura por nomes
        if not achou and doc.noun_chunks:
            for noun in doc.noun_chunks:
                for n_part in noun.text.split():
                    if n_part in k_titulo:
                        achou = True
                        como_achou = 'NOUNS'
         
        # Lemmanization
        if not achou:
            doc_titulo = nlp(k_titulo)
            for token_titulo in doc_titulo:
                if token_titulo.is_stop:
                    continue
                titulo_lemma = token_titulo.lemma_.lower()
                for token_pergunta in doc:
                    if token_pergunta.is_stop:
                        continue
                    if titulo_lemma == token_pergunta.lemma_.lower():
                        achou = True
                        como_achou = 'LEMMA'
        
        
        if not achou:
            if doc.ents:
                for ent in doc.ents:
                    entidade = ent.text
                    if entidade in grupo.context_ner:
                            achou = True
                            como_achou = 'NER_CTX_FILE' 
        
        
        
        
        if not achou:
            print("Não achou ", k_titulo," - ", pergunta)
            #for ent in doc.ents:
            #    print (ent.text, end=',')
            #print(' ')
            qtd_nao_achou +=1
            for token_sem_ctx in AppConstants.TOKENS_COM_FALTA_CONTEXTO:
                if token_sem_ctx in pergunta:
                    estatistica_busca['PERGUNTA_SEM_CONTEXTO'] += 1
        else:
            #print("Achou - ", como_achou,"-", k_titulo," - ", pergunta)
            qtd_achou +=1
            estatistica_busca[como_achou] += 1
        
print ('Total:', total)
print('Achou', qtd_achou)
print('Não Achou', qtd_nao_achou)
print(estatistica_busca)

Não achou  abraham lincoln  -  Did his mother die of pneumonia?
Não achou  amedeo avogadro  -  Who is most noted for his contributions to the theory of molarity and molecular weight?
Não achou  amedeo avogadro  -  When did he publish another memoria?
Não achou  amedeo avogadro  -  Is he most noted for his contributions to the theory of molarity and molecular weight?
Não achou  amedeo avogadro  -  Did the scientific community not reserve great attention to his theory ?
Não achou  amedeo avogadro  -  Hence, can relative molecular  masses be calculated from the masses  of gas samples?
Não achou  amedeo avogadro  -  Did he publish his work?
Não achou  amedeo avogadro  -  Hence , can relative molecular masses be calculated from the masses of gas samples ?
Não achou  anders celsius  -  Who determined the dependence of the boiling of water with atmospheric pressure?
Não achou  anders celsius  -  What is named after him?
Não achou  anders celsius  -  Had his thermometer 100 for the freezing po

In [25]:
x = nlp('What areas do the Grevy\'s Zebras inhabit?')
for t in x:
    print(t.text, t.lemma_.lower())

What what
areas area
do do
the the
Grevy grevy
's 's
Zebras zebras
inhabit inhabit
? ?
